**Author:** Elias Rafael Ruiz-Morales

**Institution:** Wellcome Sanger institute

**August, 2023**


---

# Integration of single-nuclei data using scVI

snRNA-Seq of Placental explants

In [1]:
from __future__ import print_function
import torch

import sys, os
data_type = 'float32'
os.environ["THEANO_FLAGS"] = 'device=cuda,floatX=' + data_type + ',force_device=True'
sys.path.insert(1, './')

In [2]:
# Seed for reproducibility
import numpy as np
import pandas as pd
import scanpy as sc
from typing import Tuple

# scVI imports
import scvi
from scvi.dataset import AnnDatasetFromAnnData
from scvi.inference import UnsupervisedTrainer
from scvi.models.vae import VAE

torch.manual_seed(0)
np.random.seed(0)
sc.settings.verbosity = 0  # verbosity: errors (0), warnings (1), info (2), hints (3)



def MovePlots(plotpattern, subplotdir):
    os.system('mkdir -p '+str(sc.settings.figdir)+'/'+subplotdir)
    os.system('mv '+str(sc.settings.figdir)+'/*'+plotpattern+'** '+str(sc.settings.figdir)+'/'+subplotdir)

sc.settings.verbosity = 3  # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.settings.figdir = '../results/scVI/'
sc.logging.print_versions()
sc.settings.set_figure_params(dpi=80)  # low dpi (dots per inch) yields small inline figures

sys.executable

-----
anndata     0.7.6
scanpy      1.7.0
sinfo       0.3.1
-----
PIL                 8.3.0
absl                NA
anndata             0.7.6
anyio               NA
attr                21.2.0
babel               2.9.1
backcall            0.2.0
beta_ufunc          NA
binom_ufunc         NA
brotli              NA
cairo               1.20.1
certifi             2021.05.30
cffi                1.14.4
chardet             4.0.0
colorama            0.4.4
cycler              0.10.0
cython_runtime      NA
dateutil            2.8.1
debugpy             1.3.0
decorator           5.0.9
defusedxml          0.7.1
dunamai             1.5.5
fsspec              2021.06.1
get_version         3.2
google              NA
h5py                3.3.0
idna                2.10
igraph              0.8.3
ipykernel           6.0.0
ipython_genutils    0.2.0
ipywidgets          7.6.3
jedi                0.18.0
jinja2              3.0.1
joblib              1.0.1
json5               NA
jsonschema          3.2.0
jupyter_ser

'/opt/conda/envs/scvi-singularity/bin/python'

In [3]:
def identityTransfer(adata_CellsNewID, adata, field='identity'):
    
    '''
    Function to transfer the value in a .obs column, modifying only the cells included in adata_CellsNewID
    
    Parameters:
        adata_CellsNewID: anndata object with cells. These should have in their .obs[field] the new values to be transfer to adata
        adata: anndata object to modify. The value in .obs[field] will be modifies for all the cells in adata_CellsNewID
        field: column name in .obs to operate in.
        
     Return:
        anndata object with the metadata modified
        
    '''
    
    #converting categorical into strings to introduce new categories
    adata.obs[field]=adata.obs[field].astype("string")


    #assigning the new categories to the cells in adata
    for cell in adata_CellsNewID.obs.index:
        adata.obs[field][cell] = adata_CellsNewID.obs[field][cell]

    #Returning strings into categorical 
    adata.obs[field]=adata.obs[field].astype("category")
    
    return(adata)

In [4]:
def loadCounts( meta, data_dir):
    '''
    meta: pandas dataframe with metadata.
    data_dir: path to the data
    
    '''

    for sample in meta.index:
        print(sample)
        # Load 10x data as AnnData

        holder.append(sc.read_10x_mtx(data_dir+sample+'/filtered_feature_bc_matrix/',var_names='gene_symbols',cache=True)) 
        print('Original number of cells: {:d}'.format(holder[-1].n_obs))

        holder[-1].var_names_make_unique()
        #holder[-1].obs_names = [sample+'_'+i.split('-')[0] for i in holder[-1].obs_names]


        # add in metadata
        holder[-1].obs['sample'] = sample
        for val in meta.columns:
            holder[-1].obs[val] = meta[val][sample]

        # Extract mitochondial genes
        mito_genes = [name for name in holder[-1].var_names if name.startswith('MT-')]
        #for each cell compute fraction of counts in mito genes vs. all genes
        #the `.A1` is only necessary, as X is sparse - it transform to a dense array after summing
        holder[-1].obs['percent_mito'] = np.sum(
             holder[-1][:, mito_genes].X, axis=1).A1 / np.sum(holder[-1].X, axis=1).A1

        #add the total counts per cell as observations-annotation to adata
        holder[-1].obs['n_counts'] = holder[-1].X.sum(axis=1).A1

        print('Total number of cells: {:d}'.format(holder[-1].n_obs))
        print('Total number of genes: {:d}'.format(holder[-1].n_vars))

### Loading denoised data

In [5]:
adata=sc.read('../data/snRNA1_counts_preprocessed_All_adata.h5ad')

In [6]:
adata

AnnData object with n_obs × n_vars = 66209 × 36601
    obs: 'sample', 'stage', 'donor', 'hpi', 'infection', 'percent_mito', 'n_counts', 'batch', 'n_genes', 'sample_barcode', 'assignment_SoC', 'donor_deconv', 'donor_souporcell', 'donor_id', 'scrublet_score', 'scrublet_cluster_score', 'zscore', 'bh_pval', 'bonf_pval', 'is_doublet', 'infection_stage', 'S_score', 'G2M_score', 'phase'
    var: 'gene_ids', 'feature_types', 'mean', 'std', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm'
    uns: 'donor_id_colors', 'hvg', 'infection_colors', 'sample_colors'
    layers: 'raw_counts'

In [7]:
adata.layers['raw_counts']= adata.X.copy()

### Compute the scVI latent space

Based on the scVI documentation.

In [8]:
np.unique(adata.obs['donor_id'])

array(['donor_doublets', 'snDonor_Pf0', 'snDonor_Pf1', 'snDonor_Pf2',
       'snDonor_Pf3', 'snDonor_Pf5', 'snDonor_Pf6'], dtype=object)

In [9]:
adata.obs['sample']

Pla_HDBR11923126_AAACCCAAGACCAACG    Pla_HDBR11923126
Pla_HDBR11923126_AAACCCAAGAGAGGGC    Pla_HDBR11923126
Pla_HDBR11923126_AAACCCAAGCGAACTG    Pla_HDBR11923126
Pla_HDBR11923126_AAACCCAAGGATTTGA    Pla_HDBR11923126
Pla_HDBR11923126_AAACCCAAGTACTGTC    Pla_HDBR11923126
                                           ...       
Pla_HDBR13661578_TTTCATGGTATGCTTG    Pla_HDBR13661578
Pla_HDBR13661578_TTTCCTCAGTCCTGTA    Pla_HDBR13661578
Pla_HDBR13661578_TTTGATCTCCAGCCTT    Pla_HDBR13661578
Pla_HDBR13661578_TTTGGAGAGAGCATAT    Pla_HDBR13661578
Pla_HDBR13661578_TTTGTTGGTTAAGTCC    Pla_HDBR13661578
Name: sample, Length: 66209, dtype: category
Categories (8, object): ['Pla_HDBR11923126', 'Pla_HDBR11923127', 'Pla_HDBR13026230', 'Pla_HDBR13026231', 'Pla_HDBR13026232', 'Pla_HDBR13026233', 'Pla_HDBR13661577', 'Pla_HDBR13661578']

In [10]:
scvi.data.setup_anndata(
    adata,
    layer="raw_counts",
    batch_key='donor_id', #samples as a batch
    #donor as a covariate of the cells
    categorical_covariate_keys=["sample"] 
    #categorical_covariate_keys=['donor_souporcell',], #
    #continuous_covariate_keys=[""]
)

INFO     Using batches from adata.obs["donor_id"]                                            


Using batches from adata.obs["donor_id"]


INFO     No label_key inputted, assuming all cells have same label                           


No label_key inputted, assuming all cells have same label


INFO     Using data from adata.layers["raw_counts"]                                          


...

Computing library size prior per batch


INFO     Successfully registered anndata object containing 66209 cells, 36601 vars, 7        
         batches, 1 labels, and 0 proteins. Also registered 1 extra categorical covariates   
         and 0 extra continuous covariates.                                                  


Successfully registered anndata object containing 66209 cells, 36601 vars, 7 batches, 1 labels, and 0 proteins. Also registered 1 extra categorical covariates and 0 extra continuous covariates.


INFO     Please do not further modify adata until model is trained.                          


Please do not further modify adata until model is trained.


In [11]:
#---- check #layers

In [12]:
models = {}

# let's try a few values
n_latent_values = [15, 20, 30]

for n_latent_value in n_latent_values:
    print('n_latent_value', n_latent_value)
    models[n_latent_value] = scvi.model.SCVI(adata, n_latent = n_latent_value)

n_latent_value 15
n_latent_value 20
n_latent_value 30


In [13]:
models[30]

SCVI Model with the following params: 
n_hidden: 128, n_latent: 30, n_layers: 1, dropout_rate: 0.1, dispersion: gene, 
gene_likelihood: zinb, latent_distribution: normal
Training status: Not Trained

To print summary of associated AnnData, use: scvi.data.view_anndata_setup(model.adata)

In [14]:
latent_representations = {}

for n_latent_value in n_latent_values:
    print('training model for n_latent_value:', n_latent_value)
    models[n_latent_value].train()
    
    latent_representations[n_latent_value] = models[n_latent_value].get_latent_representation()
    
    adata.obsm["X_scVI_n_latent_" + str(n_latent_value)] = latent_representations[n_latent_value]
    
    curr_df = pd.DataFrame(adata.obsm["X_scVI_n_latent_" + str(n_latent_value)])
    curr_df.to_csv('../results/scVI/20230924_obsm_with_scVI_latent_representation_n_' + str(n_latent_value) + '_allSamples.csv')

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


training model for n_latent_value: 15
Epoch 121/121: 100%|████████████████████████████████████████| 121/121 [25:04<00:00, 12.43s/it, loss=7.86e+03, v_num=1]


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


training model for n_latent_value: 20
Epoch 121/121: 100%|████████████████████████████████████████| 121/121 [25:06<00:00, 12.45s/it, loss=7.83e+03, v_num=1]


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


training model for n_latent_value: 30
Epoch 121/121: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 121/121 [25:09<00:00, 12.47s/it, loss=7.86e+03, v_num=1]


In [15]:
#adata.X= adata.layers["denoised"].copy()

In [16]:
adata.write('../results/scVI/snRNA2_scVIintegrated_AllDataRaw.h5ad')

In [17]:
adata.var["feature_types"].value_counts()


Gene Expression    36601
Name: feature_types, dtype: int64